# Black-Schole-Merton option pricing model

This is the BSM model to price european call and put options, and equity forward contract, with continously paid dividend.

## Table of Contents

#### 1.BSM model to price European call and put options with dividend
#### 2. Equity forward contract pricing model with dividend
#### 3. Delta  and Gamma
#### 4. Example Calculation
#### 5. Call option of 2 indexes

In [56]:
import math
from scipy.stats import norm

In [57]:
C = 'call'
P = 'put'

### 1.BSM model to price European call and put options with dividend

In [58]:
#############################################################################
##### This function is intended to calculate price of European options, #####
##### which the dividend yield is a contant rate, ###########################
#####interest rate is countinously compounded. ##############################
#############################################################################

#########################################
##### optionType = 'call' or 'put' ######
##### S ,T ,K ,r ,q ,vol as float  ######
#########################################

def pricingOptionByBSM(optionType,S,T,K,r,q,vol): 
        
    d1 = (math.log(S / K) + (r - q + vol ** 2 / 2) * T) / (vol * math.sqrt(T))
    d2 = d1 - vol * math.sqrt(T)
    if optionType == C:
        return S * math.exp(- q * T) * norm.cdf(d1) - K * math.exp(- r * T) * norm.cdf(d2)
    elif optionType == P:
        return K * math.exp(- r * T) * norm.cdf(- d2) - S * math.exp(- q * T) * norm.cdf(- d1)
    else:
        print('Invalid option type')
        return None
        
########################################################
##### This function implements the put-call parity #####
##### to validate the option pricing model above.  #####
########################################################
        
def validateByPutCallParity(call,put,S,y,T,K,r):
    
    leftSide = call + K * math.exp(- r * T)
    rightSide = put + S * math.exp(- y * T)
    if round(leftSide,4) == round(rightSide,4):
        return 'satisfied'
    return 'not sarisfied'   

### 2. Equity forward contract pricing model with dividend

In [59]:
##################################################################
##### This function calculates the equity forward contract. ######
##################################################################

def pricingForward(S,T,K,r,q):

    return K * math.exp(- q * T) - S * math.exp((r - q) * T) * math.exp(- r * T)

### 3. Delta  and Gamma

In [60]:
#################################################################
##### This function calculates the delta of European option #####
#################################################################

def calculateDelta(optionType,S,T,K,r,q,vol):
    
    d1 = (math.log(S / K) + (r - q + vol ** 2 / 2) * T) / (vol * math.sqrt(T))
    if optionType == C:
        return math.exp(- q * T) * norm.cdf(d1)
    elif optionType == P:
        return - math.exp(- q * T) * norm.cdf(- d1)
    else:
        print('Invalid option type')
        return None
    
#################################################################
##### This function calculates the gamma of European option #####
#################################################################
    
def calculateGamma(optionType,S,T,K,r,q,vol):
    
    d1 = (math.log(S / K) + (r - q + vol ** 2 / 2) * T) / (vol * math.sqrt(T))
    if optionType == C or optionType == P:
        return (math.exp(- q * T) * norm.pdf(d1)) / (S * vol * math.sqrt(T))
    print('Invalid option type')
    return None

### 4. Example Calculation

In [61]:
#Input example data

S = 3660 # spot price
T = 3/12 # time to maturity
K = 3800 # strike price
r = 0.01 # interest rate
q = 0.012 # dividend yield
vol = 0.25 # volatility

#call price

callPrice = pricingOptionByBSM(C,S,T,K,r,q,vol)
print(f"The call price is: {callPrice}.")

#put price

putPrice = pricingOptionByBSM(P,S,T,K,r,q,vol)
print(f"The put price is: {putPrice}.")

#forward price

forwardValue = pricingForward(S,T,K,r,q)
print(f"The value of the forward contract is: {forwardValue}.")

#put-call parity

putcallparity = validateByPutCallParity(callPrice,putPrice,S,q,T,K,r)
print(f"Put call parity: {putcallparity}.")

#put option delta

delta = calculateDelta(P,S,T,K,r,q,vol)
print(f"The Delta of this put option is {delta}.")


#put option gamma

gamma = calculateGamma(P,S,T,K,r,q,vol)
print(f"The Gamma of this put option is {gamma}.")


The call price is: 123.14347522330763.
The put price is: 264.6188867913115.
The value of the forward contract is: 139.58062937047225.
Put call parity: satisfied.
The Delta of this put option is -0.5937498062939109.
The Gamma of this put option is 0.0008443446032102217.


### 5. Call option of 2 indexes

To calculate the price of a basket option where the underlying assets are S&P 500 and FTSE 100, there are multiple methods available. In order to eliminate the confusion of using different parameters for different methods, the Longstaff-Schwartz model would be sepcified, we need: 

                -weight of investment in S&P 500 and FTSE 100 separately,
                -spot price of the underlying assets, 
                -strike price of the call option, 
                -time to maturity,
                -annual volatilities of each asset, 
                -correlations between S&P 500 and FTSE 100, 
                -risk free interest rate,
                -number of simulation periods
                -number of trials